<a href="https://colab.research.google.com/github/parmidamardi/CDM-TA-3/blob/main/CDM_TA_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
np.random.seed(42)

A = np.array([
    [1, 2.0],
    [1, 3.0],
    [1, 4.5],
    [1, 5.5]
])

y = np.array([
    [3.1],
    [4.9],
    [8.2],
    [10.1]
])

m = len(y)

In [3]:
#step 1.1
AtA = A.T @ A
AtA_inv = np.linalg.inv(AtA)
Aty = A.T @ y
theta = AtA_inv @ Aty

print("Theta:\n", theta)

Theta:
 [[-1.04137931]
 [ 2.03103448]]


In [6]:
#step 1.2: BGD
learning_rate = 0.01
n_iterations = 1000

theta_bgd = np.zeros((2,1))
for i in range(n_iterations):
    gradient = (1/m) * A.T @ (A @ theta_bgd - y)
    theta_bgd = theta_bgd - learning_rate * gradient
print("Theta from BGD:\n", theta_bgd)

Theta from BGD:
 [[-0.55217831]
 [ 1.9146829 ]]


In [8]:
#Own test: increasing the number of iterations
learning_rate = 0.01
n_iterations = 5000

theta_bgd_test = np.zeros((2,1))
for i in range(n_iterations):
    gradient = (1/m) * A.T @ (A @ theta_bgd_test - y)
    theta_bgd_test = theta_bgd_test - learning_rate * gradient
print("Theta from BGD with :\n", theta_bgd_test)

Theta from BGD with :
 [[-1.03491408]
 [ 2.02949679]]


In [9]:
#step 1.3
U, S, VT = np.linalg.svd(A, full_matrices=False)
S_inv = np.diag(1 / S)
A_pseudo_inv = VT.T @ S_inv @ U.T
theta_svd = A_pseudo_inv @ y
print("Theta from SVD pseudoinverse:\n", theta_svd)


Theta from SVD pseudoinverse:
 [[-1.04137931]
 [ 2.03103448]]


In [10]:
#step 1.4
noise = np.random.rand(m, 1) * 0.0001
A_collinear = np.hstack((A, A[:, [1]] + noise))

In [12]:
AtA = A_collinear.T @ A_collinear
AtA_inv = np.linalg.inv(AtA)
Aty = A_collinear.T @ y
theta_normal_col = AtA_inv @ Aty

print("Theta:\n", theta_normal_col)

Theta:
 [[-8.53782625e-01]
 [ 3.43827452e+03]
 [-3.43623267e+03]]


In [13]:
U, S, VT = np.linalg.svd(A_collinear, full_matrices=False)
S_inv = np.diag(1 / S)
A_pseudo_inv = VT.T @ S_inv @ U.T
theta_svd_col = A_pseudo_inv @ y
print("Theta from SVD pseudoinverse:\n", theta_svd_col)

Theta from SVD pseudoinverse:
 [[-8.53775599e-01]
 [ 3.43826298e+03]
 [-3.43622113e+03]]
